In [ ]:
import os
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Calculate the porosity
BD_om = 1.3
BD_minerals = 2.71
BD_gravels = 2.80
vf_pores_gravels = 0.24


path_out = os.path.join(os.environ['PROJDIR'], 'Soil_Moisture_v2', 'intermediate',
                        'map_predictors', 'hwsd2')
for layer in [f'D{i}' for i in range(1, 8)]:
    data_list = {}
    for var in ['COARSE', 'SAND', 'SILT', 'CLAY', 'BULK', 'ORG_CARBON']:
        h = rio.open(os.path.join(path_out, f'{var}_{layer}.shp'))
        data_list[var] = h.read(1, masked = True)
        data_list[var].mask = np.logical_or(data_list[var].mask, (data_list[var].data < 0))

        profile = dict(h.profile)

        h.close()

    # volumetric fraction
    vf_gravels_s = data_list['COARSE'] * (1 - vf_pores_gravels) / 100
    # weight fraction of SOM within fine earth
    wf_om_fine_earth = data_list['ORG_CARBON'] * 1.724 / 100
    # volume fraction of SOM within fine earth
    vf_om_fine_earth = np.ma.minimum(wf_om_fine_earth * data_list['BULK'] / BD_om, 1)

    # Bulk density of soil (GRAVELS + MINERALS + ORGANIC MATTER)
    # BD is the BULK DENSITY OF FINE EARTH (MINERALS + ORGANIC MATTER)
    BD_avg = (1 - vf_gravels_s/(1 - vf_pores_gravels)) * data_list['BULK'] + vf_gravels_s * BD_gravels

    # Mass fraction of gravels
    wf_gravels_s = vf_gravels_s * BD_gravels / BD_avg
    wf_sand_s = data_list['SAND'] / 100 * (1 - wf_om_fine_earth) * (1 - wf_gravels_s)
    wf_silt_s = data_list['SILT'] / 100 * (1 - wf_om_fine_earth) * (1 - wf_gravels_s)
    wf_clay_s = data_list['CLAY'] / 100 * (1 - wf_om_fine_earth) * (1 - wf_gravels_s)
    wf_om_s = wf_om_fine_earth * (1 - wf_gravels_s)

    # Volumetric fraction of soil constituents
    vf_sand_s = wf_sand_s * BD_avg / BD_minerals
    vf_silt_s = wf_silt_s * BD_avg / BD_minerals
    vf_clay_s = wf_clay_s * BD_avg / BD_minerals
    vf_om_s = wf_om_s * BD_avg / BD_om

    # Particle density of soil (minerals + organic matter + gravels)
    BD_particle_inverse = wf_gravels_s / BD_gravels + \
        (1 - wf_gravels_s) * ((1 - wf_om_fine_earth)/BD_minerals + wf_om_fine_earth/BD_om)
    BD_particle = 1 / BD_particle_inverse

    # Porosity of soil
    vf_pores_s = 1 - BD_avg * BD_particle_inverse

    # Save to file
    profile['driver'] = 'GTiff'
    profile['height'] = vf_pores_s.shape[0]
    profile['dtype'] = np.float32
    profile['nodata'] = 1e20
    file_out = os.path.join(path_out, f'porosity_{layer}.tif')
    dst = rio.open(file_out, 'w', **profile)
    dst.write(vf_pores_s, 1)
    dst.close()

In [ ]:
plt.imshow(vf_pores_s)
plt.colorbar()